In [ ]:
import pandas as pd
import datetime
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Variables globales
inventario = pd.DataFrame()
ventas = pd.DataFrame()

def cargar_inventario():
    """Carga el inventario inicial desde el archivo Excel"""
    global inventario
    try:
        inventario = pd.read_excel('Inventario_Tienda_Barrio.xlsx')
        print("\nInventario cargado correctamente.")
    except FileNotFoundError:
        print("\nError: No se encontró el archivo 'Inventario_Tienda_Barrio.xlsx'")
        inventario = pd.DataFrame(columns=['Categoría', 'Nombre', 'Código', 'Precio de Compra (COP)',
                                          'Precio Unitario (COP)', 'Cantidad en Stock'])
    except Exception as e:
        print(f"\nError al cargar el inventario: {e}")

def cargar_ventas():
    """Carga el historial de ventas desde el archivo CSV"""
    global ventas
    try:
        if os.path.exists('ventas.csv'):
            ventas = pd.read_csv('ventas.csv')
            # Convertir la columna de fecha a datetime
            ventas['Fecha y hora'] = pd.to_datetime(ventas['Fecha y hora'])
        else:
            ventas = pd.DataFrame(columns=['Fecha y hora', 'Código del producto', 'Nombre del producto',
                                          'Categoría', 'Cantidad vendida', 'Precio unitario',
                                          'Precio de compra', 'Subtotal', 'Utilidad'])
    except Exception as e:
        print(f"\nError al cargar ventas: {e}")

def guardar_ventas():
    """Guarda el historial de ventas en el archivo CSV"""
    try:
        ventas.to_csv('ventas.csv', index=False)
    except Exception as e:
        print(f"\nError al guardar ventas: {e}")

def mostrar_menu_principal():
    """Muestra el menú principal y maneja las opciones"""
    while True:
        print("\n=== MENÚ PRINCIPAL ===")
        print("1. Gestión de Productos")
        print("2. Registro de Ventas")
        print("3. Consultas y Reportes")
        print("4. Análisis Exploratorio de Datos (EDA)")
        print("5. Salir")

        opcion = input("\nSeleccione una opción: ")

        if opcion == '1':
            gestion_productos()
        elif opcion == '2':
            registro_ventas()
        elif opcion == '3':
            consultas_reportes()
        elif opcion == '4':
            analisis_datos()
        elif opcion == '5':
            print("\nSaliendo del sistema...")
            break
        else:
            print("\nOpción no válida. Intente nuevamente.")

def gestion_productos():
    """Menú para la gestión de productos"""
    while True:
        print("\n=== GESTIÓN DE PRODUCTOS ===")
        print("1. Agregar producto")
        print("2. Modificar producto")
        print("3. Eliminar producto")
        print("4. Listar productos por categoría")
        print("5. Volver al menú principal")

        opcion = input("\nSeleccione una opción: ")

        if opcion == '1':
            agregar_producto()
        elif opcion == '2':
            modificar_producto()
        elif opcion == '3':
            eliminar_producto()
        elif opcion == '4':
            listar_productos()
        elif opcion == '5':
            break
        else:
            print("\nOpción no válida. Intente nuevamente.")

def agregar_producto():
    """Agrega un nuevo producto al inventario (en memoria)"""
    global inventario  # Mover esta línea al principio de la función
    print("\n=== AGREGAR PRODUCTO ===")

    try:
        categoria = input("Categoría: ")
        nombre = input("Nombre del producto: ")
        codigo = int(input("Código: "))

        # Verificar si el código ya existe
        if codigo in inventario['Código'].values:
            print("\nError: Ya existe un producto con ese código.")
            return

        precio_compra = float(input("Precio de compra (COP): "))
        precio_venta = float(input("Precio unitario de venta (COP): "))
        stock = int(input("Cantidad en stock: "))

        nuevo_producto = {
            'Categoría': categoria,
            'Nombre': nombre,
            'Código': codigo,
            'Precio de Compra (COP)': precio_compra,
            'Precio Unitario (COP)': precio_venta,
            'Cantidad en Stock': stock
        }

        inventario = pd.concat([inventario, pd.DataFrame([nuevo_producto])], ignore_index=True)
        print("\nProducto agregado correctamente.")

    except ValueError:
        print("\nError: Ingrese valores numéricos válidos para código, precios y stock.")
    except Exception as e:
        print(f"\nError al agregar producto: {e}")

def modificar_producto():
    """Modifica los datos de un producto existente (en memoria)"""
    global inventario  # Esta línea debe ir al principio de la función
    print("\n=== MODIFICAR PRODUCTO ===")

    if inventario.empty:
        print("\nNo hay productos en el inventario.")
        return

    try:
        codigo = int(input("Ingrese el código del producto a modificar: "))

        if codigo not in inventario['Código'].values:
            print("\nError: No existe un producto con ese código.")
            return

        idx = inventario[inventario['Código'] == codigo].index[0]
        producto = inventario.loc[idx]

        print("\nDatos actuales del producto:")
        print(f"Categoría: {producto['Categoría']}")
        print(f"Nombre: {producto['Nombre']}")
        print(f"Código: {producto['Código']}")
        print(f"Precio de compra: {producto['Precio de Compra (COP)']}")
        print(f"Precio de venta: {producto['Precio Unitario (COP)']}")
        print(f"Stock: {producto['Cantidad en Stock']}")

        print("\nIngrese los nuevos datos (deje en blanco para mantener el valor actual):")

        categoria = input(f"Categoría [{producto['Categoría']}]: ") or producto['Categoría']
        nombre = input(f"Nombre [{producto['Nombre']}]: ") or producto['Nombre']

        precio_compra = input(f"Precio de compra [{producto['Precio de Compra (COP)']}]: ")
        precio_compra = float(precio_compra) if precio_compra else producto['Precio de Compra (COP)']

        precio_venta = input(f"Precio de venta [{producto['Precio Unitario (COP)']}]: ")
        precio_venta = float(precio_venta) if precio_venta else producto['Precio Unitario (COP)']

        stock = input(f"Stock [{producto['Cantidad en Stock']}]: ")
        stock = int(stock) if stock else producto['Cantidad en Stock']

        inventario.at[idx, 'Categoría'] = categoria
        inventario.at[idx, 'Nombre'] = nombre
        inventario.at[idx, 'Precio de Compra (COP)'] = precio_compra
        inventario.at[idx, 'Precio Unitario (COP)'] = precio_venta
        inventario.at[idx, 'Cantidad en Stock'] = stock

        print("\nProducto modificado correctamente.")

    except ValueError:
        print("\nError: Ingrese valores numéricos válidos para precios y stock.")
    except Exception as e:
        print(f"\nError al modificar producto: {e}")

def eliminar_producto():
    """Elimina un producto del inventario (en memoria)"""
    global inventario  # Declaración global al inicio

    print("\n=== ELIMINAR PRODUCTO ===")

    if inventario.empty:
        print("\nNo hay productos en el inventario.")
        return

    try:
        codigo = int(input("Ingrese el código del producto a eliminar: "))

        if codigo not in inventario['Código'].values:
            print("\nError: No existe un producto con ese código.")
            return

        producto = inventario[inventario['Código'] == codigo].iloc[0]
        print(f"\nProducto a eliminar: {producto['Nombre']} ({producto['Categoría']})")

        confirmacion = input("\n¿Está seguro que desea eliminar este producto? (s/n): ").lower()

        if confirmacion == 's':
            inventario = inventario[inventario['Código'] != codigo]
            print("\nProducto eliminado correctamente.")
        else:
            print("\nOperación cancelada.")

    except ValueError:
        print("\nError: Ingrese un código numérico válido.")
    except Exception as e:
        print(f"\nError al eliminar producto: {e}")

def listar_productos():
    """Lista todos los productos agrupados por categoría"""
    print("\n=== LISTADO DE PRODUCTOS POR CATEGORÍA ===")

    if inventario.empty:
        print("\nNo hay productos en el inventario.")
        return

    for categoria, grupo in inventario.groupby('Categoría'):
        print(f"\n--- {categoria.upper()} ---")
        print(grupo[['Nombre', 'Código', 'Precio Unitario (COP)', 'Cantidad en Stock']].to_string(index=False))

    input("\nPresione Enter para continuar...")

def registro_ventas():
    """Registra una nueva venta"""
    print("\n=== REGISTRO DE VENTA ===")

    if inventario.empty:
        print("\nNo hay productos en el inventario para vender.")
        return

    productos_vendidos = []
    continuar = True

    while continuar:
        try:
            listar_productos()
            codigo = int(input("\nIngrese el código del producto a vender (0 para terminar): "))

            if codigo == 0:
                continuar = False
                continue

            if codigo not in inventario['Código'].values:
                print("\nError: No existe un producto con ese código.")
                continue

            producto = inventario[inventario['Código'] == codigo].iloc[0]
            print(f"\nProducto seleccionado: {producto['Nombre']}")
            print(f"Precio unitario: {producto['Precio Unitario (COP)']} COP")
            print(f"Stock disponible: {producto['Cantidad en Stock']}")

            cantidad = int(input("Ingrese la cantidad a vender: "))

            if cantidad <= 0:
                print("\nError: La cantidad debe ser mayor que cero.")
                continue

            if cantidad > producto['Cantidad en Stock']:
                print("\nError: No hay suficiente stock para esta venta.")
                continue

            subtotal = cantidad * producto['Precio Unitario (COP)']
            utilidad = cantidad * (producto['Precio Unitario (COP)'] - producto['Precio de Compra (COP)'])

            productos_vendidos.append({
                'codigo': codigo,
                'nombre': producto['Nombre'],
                'categoria': producto['Categoría'],
                'cantidad': cantidad,
                'precio_unitario': producto['Precio Unitario (COP)'],
                'precio_compra': producto['Precio de Compra (COP)'],
                'subtotal': subtotal,
                'utilidad': utilidad
            })

            print(f"\nProducto agregado a la venta. Subtotal: {subtotal} COP")

            otro = input("\n¿Desea agregar otro producto? (s/n): ").lower()
            if otro != 's':
                continuar = False

        except ValueError:
            print("\nError: Ingrese valores numéricos válidos.")
        except Exception as e:
            print(f"\nError al registrar venta: {e}")

    if not productos_vendidos:
        print("\nNo se registró ninguna venta.")
        return

    # Mostrar resumen de la venta
    mostrar_resumen_venta(productos_vendidos)

    # Confirmar venta
    confirmar = input("\n¿Confirmar la venta? (s/n): ").lower()

    if confirmar == 's':
        registrar_venta(productos_vendidos)
        print("\nVenta registrada correctamente.")
    else:
        print("\nVenta cancelada.")

def registrar_venta(productos):
    """Registra la venta en memoria y actualiza el stock"""
    global inventario, ventas
    fecha_hora = datetime.datetime.now()

    # Actualizar stock en inventario
    for p in productos:
        idx = inventario[inventario['Código'] == p['codigo']].index[0]
        inventario.at[idx, 'Cantidad en Stock'] -= p['cantidad']

    # Crear DataFrame con las nuevas ventas
    nuevas_ventas = pd.DataFrame([{
        'Fecha y hora': fecha_hora,
        'Código del producto': p['codigo'],
        'Nombre del producto': p['nombre'],
        'Categoría': p['categoria'],
        'Cantidad vendida': p['cantidad'],
        'Precio unitario': p['precio_unitario'],
        'Precio de compra': p['precio_compra'],
        'Subtotal': p['subtotal'],
        'Utilidad': p['utilidad']
    } for p in productos])

    # Concatenar solo si hay datos válidos
    if not ventas.empty:
        ventas = pd.concat([ventas, nuevas_ventas], ignore_index=True, sort=False)
    else:
        ventas = nuevas_ventas

    guardar_ventas()

def mostrar_resumen_venta(productos):
    """Muestra un resumen de la venta en formato de factura"""
    fecha_hora = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    total = sum(p['subtotal'] for p in productos)
    total_utilidad = sum(p['utilidad'] for p in productos)

    print("\n=== FACTURA DE VENTA ===")
    print(f"Fecha: {fecha_hora}")
    print("\nDetalle:")
    print("-" * 60)
    print(f"{'Producto':<20} {'Cant.':>6} {'P.Unit.':>10} {'Subtotal':>10}")
    print("-" * 60)

    for p in productos:
        print(f"{p['nombre'][:18]:<20} {p['cantidad']:>6} {p['precio_unitario']:>10.2f} {p['subtotal']:>10.2f}")

    print("-" * 60)
    print(f"{'TOTAL:':<36} {total:>10.2f}")
    print(f"{'Utilidad estimada:':<36} {total_utilidad:>10.2f}")
    print("-" * 60)

def registrar_venta(productos):
    """Registra la venta en memoria y actualiza el stock"""
    global inventario, ventas
    fecha_hora = datetime.datetime.now()

    # Actualizar stock en inventario
    for p in productos:
        idx = inventario[inventario['Código'] == p['codigo']].index[0]
        inventario.at[idx, 'Cantidad en Stock'] -= p['cantidad']

    # Agregar venta al historial
    nuevas_ventas = []

    for p in productos:
        nueva_venta = {
            'Fecha y hora': fecha_hora,
            'Código del producto': p['codigo'],
            'Nombre del producto': p['nombre'],
            'Categoría': p['categoria'],
            'Cantidad vendida': p['cantidad'],
            'Precio unitario': p['precio_unitario'],
            'Precio de compra': p['precio_compra'],
            'Subtotal': p['subtotal'],
            'Utilidad': p['utilidad']
        }
        nuevas_ventas.append(nueva_venta)

    ventas = pd.concat([ventas, pd.DataFrame(nuevas_ventas)], ignore_index=True)
    guardar_ventas()

def consultas_reportes():
    """Menú para consultas y reportes"""
    while True:
        print("\n=== CONSULTAS Y REPORTES ===")
        print("1. Ver todas las ventas")
        print("2. Consultar ventas por fecha")
        print("3. Producto más vendido")
        print("4. Total de ingresos")
        print("5. Ganancia total")
        print("6. Productos con stock bajo")
        print("7. Volver al menú principal")

        opcion = input("\nSeleccione una opción: ")

        if opcion == '1':
            ver_todas_ventas()
        elif opcion == '2':
            consultar_ventas_fecha()
        elif opcion == '3':
            producto_mas_vendido()
        elif opcion == '4':
            total_ingresos()
        elif opcion == '5':
            ganancia_total()
        elif opcion == '6':
            productos_stock_bajo()
        elif opcion == '7':
            break
        else:
            print("\nOpción no válida. Intente nuevamente.")

def ver_todas_ventas():
    """Muestra todas las ventas registradas"""
    print("\n=== HISTORIAL DE VENTAS ===")

    if ventas.empty:
        print("\nNo hay ventas registradas.")
        return

    # Mostrar solo las columnas más relevantes
    print(ventas[['Fecha y hora', 'Nombre del producto', 'Cantidad vendida', 'Subtotal']].to_string(index=False))
    input("\nPresione Enter para continuar...")

def consultar_ventas_fecha():
    """Consulta ventas por fecha o rango de fechas"""
    print("\n=== CONSULTA DE VENTAS POR FECHA ===")

    if ventas.empty:
        print("\nNo hay ventas registradas.")
        return

    try:
        fecha_inicio = input("Ingrese fecha de inicio (YYYY-MM-DD) o dejar en blanco: ")
        fecha_fin = input("Ingrese fecha de fin (YYYY-MM-DD) o dejar en blanco: ")

        if not fecha_inicio and not fecha_fin:
            # Mostrar todas las ventas si no se especifican fechas
            print(ventas[['Fecha y hora', 'Nombre del producto', 'Cantidad vendida', 'Subtotal']].to_string(index=False))
        else:
            # Filtrar por fechas
            if fecha_inicio:
                fecha_inicio = pd.to_datetime(fecha_inicio)
                mask = ventas['Fecha y hora'] >= fecha_inicio
            else:
                mask = pd.Series(True, index=ventas.index)

            if fecha_fin:
                fecha_fin = pd.to_datetime(fecha_fin) + pd.Timedelta(days=1)  # Incluir todo el día
                mask = mask & (ventas['Fecha y hora'] <= fecha_fin)

            ventas_filtradas = ventas[mask]

            if ventas_filtradas.empty:
                print("\nNo hay ventas en el rango de fechas especificado.")
            else:
                print(ventas_filtradas[['Fecha y hora', 'Nombre del producto', 'Cantidad vendida', 'Subtotal']].to_string(index=False))

        input("\nPresione Enter para continuar...")
    except ValueError:
        print("\nError: Formato de fecha incorrecto. Use YYYY-MM-DD.")
    except Exception as e:
        print(f"\nError al consultar ventas: {e}")

def producto_mas_vendido():
    """Muestra el producto más vendido en cantidad"""
    print("\n=== PRODUCTO MÁS VENDIDO ===")

    if ventas.empty:
        print("\nNo hay ventas registradas.")
        return

    producto_mas_vendido = ventas.groupby('Nombre del producto')['Cantidad vendida'].sum().idxmax()
    cantidad = ventas.groupby('Nombre del producto')['Cantidad vendida'].sum().max()

    print(f"\nProducto más vendido: {producto_mas_vendido}")
    print(f"Cantidad total vendida: {cantidad}")

    input("\nPresione Enter para continuar...")

def total_ingresos():
    """Muestra el total de ingresos por ventas"""
    print("\n=== TOTAL DE INGRESOS ===")

    if ventas.empty:
        print("\nNo hay ventas registradas.")
        return

    total = ventas['Subtotal'].sum()
    print(f"\nTotal de ingresos: {total:.2f} COP")

    input("\nPresione Enter para continuar...")

def ganancia_total():
    """Muestra la ganancia total (utilidad)"""
    print("\n=== GANANCIA TOTAL ===")

    if ventas.empty:
        print("\nNo hay ventas registradas.")
        return

    total = ventas['Utilidad'].sum()
    print(f"\nGanancia total: {total:.2f} COP")

    input("\nPresione Enter para continuar...")

def productos_stock_bajo():
    """Muestra productos con stock bajo (menos de 10 unidades)"""
    print("\n=== PRODUCTOS CON STOCK BAJO (<10 unidades) ===")

    if inventario.empty:
        print("\nNo hay productos en el inventario.")
        return

    stock_bajo = inventario[inventario['Cantidad en Stock'] < 10]

    if stock_bajo.empty:
        print("\nNo hay productos con stock bajo.")
    else:
        print(stock_bajo[['Nombre', 'Categoría', 'Cantidad en Stock']].to_string(index=False))

    input("\nPresione Enter para continuar...")

def analisis_datos():
    """Realiza análisis exploratorio de datos sobre las ventas"""
    print("\n=== ANÁLISIS EXPLORATORIO DE DATOS (EDA) ===")

    if ventas.empty:
        print("\nNo hay ventas registradas para analizar.")
        return

    while True:
        print("\n=== OPCIONES DE ANÁLISIS ===")
        print("1. Resumen estadístico")
        print("2. Productos más vendidos")
        print("3. Distribución de ventas por producto")
        print("4. Ganancias por producto")
        print("5. Serie de tiempo de ingresos")
        print("6. Volver al menú anterior")

        opcion = input("\nSeleccione una opción: ")

        if opcion == '1':
            resumen_estadistico()
        elif opcion == '2':
            productos_mas_vendidos_grafico()
        elif opcion == '3':
            distribucion_ventas()
        elif opcion == '4':
            ganancias_por_producto()
        elif opcion == '5':
            serie_tiempo_ingresos()
        elif opcion == '6':
            break
        else:
            print("\nOpción no válida. Intente nuevamente.")

def resumen_estadistico():
    """Muestra un resumen estadístico de las ventas"""
    print("\n=== RESUMEN ESTADÍSTICO ===")

    # Totales básicos
    total_ventas = len(ventas)
    total_ingresos = ventas['Subtotal'].sum()
    total_ganancias = ventas['Utilidad'].sum()

    print(f"\nTotal de ventas realizadas: {total_ventas}")
    print(f"Total de ingresos generados: {total_ingresos:.2f} COP")
    print(f"Total de ganancias: {total_ganancias:.2f} COP")

    # Productos más y menos vendidos
    ventas_por_producto = ventas.groupby('Nombre del producto')['Cantidad vendida'].sum()
    producto_mas_vendido = ventas_por_producto.idxmax()
    cantidad_mas_vendido = ventas_por_producto.max()
    producto_menos_vendido = ventas_por_producto.idxmin()
    cantidad_menos_vendido = ventas_por_producto.min()

    print(f"\nProducto más vendido: {producto_mas_vendido} ({cantidad_mas_vendido} unidades)")
    print(f"Producto menos vendido: {producto_menos_vendido} ({cantidad_menos_vendido} unidades)")

    # Promedio de ventas por producto
    promedio_ventas = ventas_por_producto.mean()
    print(f"\nPromedio de ventas por producto: {promedio_ventas:.1f} unidades")

    input("\nPresione Enter para continuar...")

def productos_mas_vendidos_grafico():
    """Muestra gráfico de barras con los productos más vendidos"""
    print("\n=== PRODUCTOS MÁS VENDIDOS ===")

    # Agrupar por producto y sumar cantidades vendidas
    ventas_por_producto = ventas.groupby('Nombre del producto')['Cantidad vendida'].sum().sort_values(ascending=False)

    # Tomar los top 10 o menos si hay menos productos
    top_n = min(10, len(ventas_por_producto))
    top_productos = ventas_por_producto.head(top_n)

    # Configurar el gráfico
    plt.figure(figsize=(10, 6))
    sns.barplot(x=top_productos.values, y=top_productos.index, palette='viridis')
    plt.title(f'Top {top_n} Productos Más Vendidos')
    plt.xlabel('Cantidad Vendida')
    plt.ylabel('Producto')

    # Mostrar valores en las barras
    for i, v in enumerate(top_productos.values):
        plt.text(v + 0.5, i, str(v), color='black', va='center')

    plt.tight_layout()
    plt.show()

    # Guardar el gráfico
    guardar = input("\n¿Desea guardar este gráfico como imagen? (s/n): ").lower()
    if guardar == 's':
        try:
            plt.savefig('productos_mas_vendidos.png')
            print("\nGráfico guardado como 'productos_mas_vendidos.png'")
        except Exception as e:
            print(f"\nError al guardar el gráfico: {e}")

    plt.close()

def distribucion_ventas():
    """Muestra histograma de frecuencia por productos"""
    print("\n=== DISTRIBUCIÓN DE VENTAS POR PRODUCTO ===")

    # Agrupar por producto y sumar cantidades vendidas
    ventas_por_producto = ventas.groupby('Nombre del producto')['Cantidad vendida'].sum()

    # Configurar el gráfico
    plt.figure(figsize=(10, 6))
    sns.histplot(ventas_por_producto, bins=15, kde=True, color='skyblue')
    plt.title('Distribución de Ventas por Producto')
    plt.xlabel('Cantidad Vendida')
    plt.ylabel('Frecuencia')
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()

    # Guardar el gráfico
    guardar = input("\n¿Desea guardar este gráfico como imagen? (s/n): ").lower()
    if guardar == 's':
        try:
            plt.savefig('distribucion_ventas.png')
            print("\nGráfico guardado como 'distribucion_ventas.png'")
        except Exception as e:
            print(f"\nError al guardar el gráfico: {e}")

    plt.close()

def ganancias_por_producto():
    """Muestra gráfico de ganancias por producto"""
    print("\n=== GANANCIAS POR PRODUCTO ===")

    # Agrupar por producto y sumar utilidades
    ganancias_por_producto = ventas.groupby('Nombre del producto')['Utilidad'].sum().sort_values(ascending=False)

    # Tomar los top 10 o menos si hay menos productos
    top_n = min(10, len(ganancias_por_producto))
    top_ganancias = ganancias_por_producto.head(top_n)

    # Configurar el gráfico
    plt.figure(figsize=(10, 6))
    sns.barplot(x=top_ganancias.values, y=top_ganancias.index, palette='magma')
    plt.title(f'Top {top_n} Productos por Ganancias')
    plt.xlabel('Ganancias (COP)')
    plt.ylabel('Producto')

    # Mostrar valores en las barras (formateados como dinero)
    for i, v in enumerate(top_ganancias.values):
        plt.text(v + 0.5, i, f"${v:,.0f}", color='black', va='center')

    plt.tight_layout()
    plt.show()

    # Guardar el gráfico
    guardar = input("\n¿Desea guardar este gráfico como imagen? (s/n): ").lower()
    if guardar == 's':
        try:
            plt.savefig('ganancias_por_producto.png')
            print("\nGráfico guardado como 'ganancias_por_producto.png'")
        except Exception as e:
            print(f"\nError al guardar el gráfico: {e}")

    plt.close()

def serie_tiempo_ingresos():
    """Muestra serie de tiempo de ingresos por día"""
    print("\n=== SERIE DE TIEMPO DE INGRESOS ===")

    if ventas.empty:
        print("\nNo hay ventas registradas para mostrar.")
        input("\nPresione Enter para volver al menú...")
        return

    # Crear copia para no modificar el original
    ventas_temp = ventas.copy()

    # Extraer la fecha (sin hora)
    ventas_temp['Fecha'] = ventas_temp['Fecha y hora'].dt.date

    # Agrupar por fecha y sumar subtotales
    ingresos_por_dia = ventas_temp.groupby('Fecha')['Subtotal'].sum()

    # Configurar el gráfico
    plt.figure(figsize=(12, 6))
    sns.lineplot(x=ingresos_por_dia.index, y=ingresos_por_dia.values, marker='o', color='green')
    plt.title('Ingresos por Día')
    plt.xlabel('Fecha')
    plt.ylabel('Ingresos (COP)')
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.7)

    # Formatear eje Y como dinero
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"${x:,.0f}"))

    plt.tight_layout()

    # Mostrar mensaje de instrucción
    print("\nLa gráfica se mostrará en una ventana emergente.")
    print("Cierre la ventana de la gráfica para continuar...")

    # Mostrar la gráfica (esto bloqueará la ejecución hasta que se cierre la ventana)
    plt.show()

    # Después de cerrar la gráfica, preguntar si guardar
    guardar = input("\n¿Desea guardar este gráfico como imagen? (s/n): ").lower()
    if guardar == 's':
        try:
            plt.savefig('ingresos_por_dia.png', bbox_inches='tight')
            print("\nGráfico guardado como 'ingresos_por_dia.png'")
        except Exception as e:
            print(f"\nError al guardar el gráfico: {e}")

    plt.close()

    # Volver al menú automáticamente después de cerrar la gráfica
    print("\nVolviendo al menú anterior...")

def main():
    """Función principal que inicia el programa"""
    print("\n=== SISTEMA DE REGISTRO DE VENTAS - TIENDA DE BARRIO ===")

    # Cargar datos iniciales
    cargar_inventario()
    cargar_ventas()

    # Mostrar menú principal
    mostrar_menu_principal()

if __name__ == "__main__":
    main()


=== SISTEMA DE REGISTRO DE VENTAS - TIENDA DE BARRIO ===

Error: No se encontró el archivo 'Inventario_Tienda_Barrio.xlsx'

=== MENÚ PRINCIPAL ===
1. Gestión de Productos
2. Registro de Ventas
3. Consultas y Reportes
4. Análisis Exploratorio de Datos (EDA)
5. Salir
